# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read file
data = pd.read_csv("../output_data/cities.csv")

trip_data_df = data.drop(columns=(['Unnamed: 0']))

#Display trip_data dataframe to confirm
trip_data_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sawtell,-30.38,153.10,64.40,93,75,7.20,AU,12/11/2020
1,Yenagoa,4.92,6.26,81.73,80,34,2.10,NG,12/11/2020
2,Hithadhoo,-0.60,73.08,82.11,76,14,5.38,MV,12/11/2020
3,Soavinandriana,-19.17,46.73,72.14,73,97,4.05,MG,12/11/2020
4,Bluff,-46.60,168.33,48.00,87,92,7.60,NZ,12/11/2020
...,...,...,...,...,...,...,...,...,...
566,Rio Grande,-32.03,-52.10,69.01,93,0,0.45,BR,12/11/2020
567,Progreso,21.28,-89.67,66.99,77,75,4.10,MX,12/11/2020
568,The Valley,18.22,-63.06,78.01,83,40,1.50,AI,12/11/2020
569,Saint Lawrence,36.06,14.20,60.80,77,75,7.20,MT,12/11/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps (import key)
from api_keys import g_key

#Configure maps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = trip_data_df[["Lat","Lng"]]

# Filla NaN values and convert to float
humidity = trip_data_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

#Add heat layer to heat map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [153]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#  * A max temperature lower than 80 degrees but higher than 70.
#  * Wind speed less than 10 mph.
#  * Zero cloudiness.
#  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


ideal_trip_data_df = trip_data_df[(trip_data_df['Max Temp'] < 80) &
                                  (trip_data_df['Max Temp'] > 70) &
                                  (trip_data_df['Wind Speed'] < 10) &
                                  (trip_data_df['Cloudiness'] == 0)].reset_index()

ideal_trip_data_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,26,Atuona,-9.80,-139.03,78.85,79,0,9.80,PF,12/11/2020
1,42,Tura,25.52,90.22,74.03,49,0,1.78,IN,12/11/2020
2,69,Sur,22.57,59.53,77.07,47,0,9.76,OM,12/11/2020
3,160,Sittwe,20.15,92.90,78.10,54,0,2.88,MM,12/11/2020
4,190,Montevideo,-34.83,-56.17,73.00,60,0,4.10,UY,12/11/2020
5,213,Mount Gambier,-37.83,140.77,71.60,35,0,7.20,AU,12/11/2020
6,308,Agadez,19.75,10.25,71.11,23,0,2.54,NE,12/11/2020
7,333,Cidreira,-30.18,-50.21,70.66,88,0,7.70,BR,12/11/2020
8,344,Suleja,9.18,7.18,75.20,78,0,1.00,NG,12/11/2020
9,371,Tezpur,26.63,92.80,77.22,53,0,0.57,IN,12/11/2020


In [154]:
#Record count of cities
len(ideal_trip_data_df)

17

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [170]:
# Set up hotel dataframe
hotel_df = ideal_trip_data_df
#temporary place holder for hotel name
hotel_df["Hotel Name"] = hotel_df["City"]
#drop previous index to establish new sequence
hotel_df = hotel_df.drop(columns='index')
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atuona,-9.80,-139.03,78.85,79,0,9.80,PF,12/11/2020,Atuona
1,Tura,25.52,90.22,74.03,49,0,1.78,IN,12/11/2020,Tura
2,Sur,22.57,59.53,77.07,47,0,9.76,OM,12/11/2020,Sur
3,Sittwe,20.15,92.90,78.10,54,0,2.88,MM,12/11/2020,Sittwe
4,Montevideo,-34.83,-56.17,73.00,60,0,4.10,UY,12/11/2020,Montevideo
5,Mount Gambier,-37.83,140.77,71.60,35,0,7.20,AU,12/11/2020,Mount Gambier
6,Agadez,19.75,10.25,71.11,23,0,2.54,NE,12/11/2020,Agadez
7,Cidreira,-30.18,-50.21,70.66,88,0,7.70,BR,12/11/2020,Cidreira
8,Suleja,9.18,7.18,75.20,78,0,1.00,NG,12/11/2020,Suleja
9,Tezpur,26.63,92.80,77.22,53,0,0.57,IN,12/11/2020,Tezpur


In [172]:
# set up lists to hold response info
hotel = []

#Set Counter
counter = 0

#Find Hotel
#    CP Hint: See https://developers.google.com/places/web-service/supported_types

#Set type
type_search = "lodging"

radius = 5000

#Build a URL using the Google Map API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through the list of cities and perform a request for hotel name on each

for city in ideal_trip_data_df["City"]:
    
    #Build a URL using the Google Map API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #Build lat long coordinate string to sync with for loop
    city_coords = str(hotel_df['Lat'][counter])+","+str(hotel_df['Lng'][counter])
    
    #Set params
    params = {
        "location": city_coords,
        "types": type_search,
        "radius": radius,
        "key": g_key
        }
    
    #Print tracking detail
    print ("Processing Record "+str(counter)+" of Set 1 | "+city)
    counter += 1
    
    try:
        response = requests.get(base_url,params).json()
        
        # Add requested city fields
        hotel.append(response["results"][0]["name"])
              
    except IndexError:
        #Print exception detail
        print("City not found, skipping")
        hotel.append(np.nan)       



Processing Record 0 of Set 1 | Atuona
Processing Record 1 of Set 1 | Tura
Processing Record 2 of Set 1 | Sur
Processing Record 3 of Set 1 | Sittwe
Processing Record 4 of Set 1 | Montevideo
Processing Record 5 of Set 1 | Mount Gambier
Processing Record 6 of Set 1 | Agadez
City not found, skipping
Processing Record 7 of Set 1 | Cidreira
Processing Record 8 of Set 1 | Suleja
Processing Record 9 of Set 1 | Tezpur
Processing Record 10 of Set 1 | Sikasso
City not found, skipping
Processing Record 11 of Set 1 | Suez
Processing Record 12 of Set 1 | Prieska
Processing Record 13 of Set 1 | Acapulco de Juárez
Processing Record 14 of Set 1 | Santo Ângelo
Processing Record 15 of Set 1 | Bilma
Processing Record 16 of Set 1 | Amoy


In [175]:
#update dataframe with the  hotel names
hotel_df['Hotel Name'] = hotel
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Atuona,-9.80,-139.03,78.85,79,0,9.80,PF,12/11/2020,Villa Enata
1,Tura,25.52,90.22,74.03,49,0,1.78,IN,12/11/2020,Hotel RIKMAN Continental
2,Sur,22.57,59.53,77.07,47,0,9.76,OM,12/11/2020,Sur Plaza Hotel
3,Sittwe,20.15,92.90,78.10,54,0,2.88,MM,12/11/2020,Hotel Memory
4,Montevideo,-34.83,-56.17,73.00,60,0,4.10,UY,12/11/2020,Besares Social Club
5,Mount Gambier,-37.83,140.77,71.60,35,0,7.20,AU,12/11/2020,Commodore on the Park
6,Agadez,19.75,10.25,71.11,23,0,2.54,NE,12/11/2020,NaN
7,Cidreira,-30.18,-50.21,70.66,88,0,7.70,BR,12/11/2020,Hotel Castelo
8,Suleja,9.18,7.18,75.20,78,0,1.00,NG,12/11/2020,Capy Motel
9,Tezpur,26.63,92.80,77.22,53,0,0.57,IN,12/11/2020,Hotel Kanyapur


In [179]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [181]:
# Add marker layer ontop of heat map

# Store latitude and longitude in locations
locations = hotel_df[["Lat","Lng"]]

# Filla NaN values and convert to float
humidity = hotel_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

#Create Marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add layers to heat map
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))